In [1]:
from scipy.integrate import quad
from numpy import *
from matplotlib.pylab import *
import matplotlib as mpl
%matplotlib inline
#from pandas.core import *
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.decomposition import PCA,FastICA

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
folder = '/mnt/data/spinemodel/conditions/'

In [3]:
#Jupyters: Proper Spine Model - Inhibition.ipynb
with open(folder+"dis_baselinev3_1000_70_65sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_1000_70_65sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)


In [4]:
dataT.keys(),vtracsST.shape

(dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0', 'Rmorph']),
 (10002, 384))

In [5]:
pddataT = pd.DataFrame(dataT)

In [6]:
labels = ('Vh','A1','Ah','Dss','Rneck','Lhead','meanDhead')
datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

labels = ('A1','Dss','Rneck','SA')
datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

In [7]:
pddataT = pd.DataFrame(data=datt,columns=labels)

In [8]:
regressor = LinearRegression()  
regressor.fit(datt, mesT[:,3])
coeforig = regressor.coef_*1.0
print(regressor.coef_)

[ 3.06790858e+00 -3.93328064e-03 -5.48712705e-05 -8.98420167e-03]


In [9]:
y_pred = regressor.predict(datt)

In [10]:
fullvarlinear = ((mesT[:,3]-y_pred)).var()
print(fullvarlinear)

0.008987657974958048


In [11]:
def LinReg_varexplain_B(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    var0 = var(y)
    #y_predt = y*0.0    
    for k in range(X.shape[1]-1):

        varts = zeros(len(idcs))
        #print(idcs)
        for iv,i in enumerate(idcs):
            
            idct = 1*array(idcs)
            idct = delete(idct,iv)
            Xb = X.values[:,array(idct)]
            
            regressor.fit(Xb, y)
            y_pred = regressor.predict(Xb)
            vart = ((y-y_pred)).var()
            
            varts[iv] = vart
        
        #print(varts)
        # We select column that affected the minimum and remove it
        sel = argmin(varts)
        i = idcs[sel]
        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    #i = idcs[0]
    vardat[-1,:] = (idcs[0],var(y))
    #print(vardat,var0)
    #hist(y-y_pred,41)

    result = {}
    #print(vardat)
    for i in range(vardat.shape[0]-1,0,-1):
        vsa = (vardat[i,1])/var0
        vs = (vardat[i-1,1])/var0
        #print(labels[int(vardat[i,0])],(vsa-vs)*100)
        result[labels[int(vardat[i,0])]] = int(10000*(vsa-vs))/10000
        
    regressor.fit(X, y)
    y_pred = regressor.predict(X)
    vart = ((y-y_pred)).var()
    vsa = (vardat[0,1])/var0
    vs = (vart)/var0
    
    #print(labels[int(vardat[0,0])],(vsa-vs)*100)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa-vs))/10000
    result['variance'] = int(10000*(var0))/10000
    return(result)

In [12]:
def LinReg_varexplain_F(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    y_predt = y*0.0
    var0 = var(y)
    minvar = var0
    for k in range(X.shape[1]-1):
        varts = zeros(len(idcs))
        for iv,i in enumerate(idcs):
            regressor.fit(X.values[:,i:(i+1)], y-y_predt)
            y_pred = regressor.predict(X.values[:,i:(i+1)])
            vart = ((y-y_predt-y_pred)).var()
            varts[iv] = vart
        sel = argmin(varts)
        i = idcs[sel]
        regressor.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = regressor.predict(X.values[:,i:(i+1)])
        y_predt += y_pred

        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    i = idcs[0]
    regressor.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = regressor.predict(X.values[:,i:(i+1)])
    vart = ((y-y_predt-y_pred)).var()
    vardat[-1,:] = (idcs[0],vart)

    result = {}

    
    #hist(y-y_predt-y_pred,41)

    #print(vardat)
    vsa = 1-(vardat[0,1])/var0
    #print(labels[int(vardat[0,0])],vsa*100,vsa)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa))/10000
    for i in range(1,vardat.shape[0]):
        #print(vardat[i-1,1],vardat[i,1])
        if relative:
            var0 = vardat[i-1,1]
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs))/10000
        else:
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs-vsa))/10000
            vsa = vs
    return(result)

In [13]:
print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))

{'Rneck': 0.2598, 'A1': 0.3477, 'Dss': 0.0007, 'SA': 0.0004, 'variance': 20.8224}
{'Rneck': 0.2598, 'A1': 0.3246, 'SA': 0.0002, 'Dss': 0.0}
{'Rneck': 0.2598, 'A1': 0.4385, 'SA': 0.0006, 'Dss': 0.0}


In [14]:
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))

{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'SA': 0.0001, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'SA': 0.0001}
{'A1': 0.8907, 'Dss': 0.5857, 'Rneck': 0.0105, 'SA': 0.004}


In [15]:
dataT.keys()

dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0', 'Rmorph'])

In [21]:
with open(folder+"dis_baselinev3_1000_70_65sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_1000_70_65sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)

labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = ['A1','Dss','Rneck']#,'meanDhead']
labelsmod = labels
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp/dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]/mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]/mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]/mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp/dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]/mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]/mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]/mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004}
{'A1': 0.8907, 'Dss': 0.5857, 'Rneck': 0.0105}
**********V In Spine head**********
{'A1': 0.602, 'Rneck': 0.2015, 'Dss': 0.0001, 'variance': 39.3462}
{'A1': 0.602, 'Rneck': 0.1881, 'Dss': 0.0003}
{'A1': 0.602, 'Rneck': 0.4727, 'Dss': 0.0018}
**********V sp/dend**********
{'Rneck': 0.9687, 'Dss': 0.0081, 'A1': 0.004, 'variance': 2.3418}
{'Rneck': 0.9687, 'Dss': 0.0079, 'A1': 0.0042}
{'Rneck': 0.9687, 'Dss': 0.2555, 'A1': 0.1818}
**********C In Spine head**********
{'A1': 0.3023, 'Rneck': 0.094, 'Dss': 0.0001, 'variance': 0.8904}
{'A1': 0.3023, 'Rneck': 0.0877, 'Dss': 0.0}
{'A1': 0.3023, 'Rneck': 0.1258, 'Dss': 0.0}

#---------------DiS!
**********V In Soma**********
{'A1': 0.7668, 'Dss': 0.1656, 'Rneck': 0.0041, 'variance': 0.1644}
{'A1': 0.7668, 'Dss': 0.1466, 'Rneck': 0.0091}
{'A1': 0.7668, 'Dss': 0.6292, 'Rneck': 0.1054}
**********V In Spine he

In [22]:
labels = ['maxDhead','Dss','Lneck','Lhead','meanDneck_trunc70pc', 'A1']
labelmorph = labels
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp/dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]/mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]/mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]/mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp/dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]/mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]/mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]/mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Lneck': 0.0003, 'Lhead': 0.0002, 'meanDneck_trunc70pc': 0.0, 'maxDhead': 0.0, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Lneck': 0.0004, 'Lhead': 0.0001, 'meanDneck_trunc70pc': 0.0, 'maxDhead': 0.0}
{'A1': 0.8907, 'Dss': 0.5857, 'Lneck': 0.0089, 'Lhead': 0.0044, 'meanDneck_trunc70pc': 0.0015, 'maxDhead': 0.0}
**********V In Spine head**********
{'A1': 0.602, 'Lneck': 0.14, 'meanDneck_trunc70pc': 0.0487, 'maxDhead': 0.0049, 'Dss': 0.0011, 'Lhead': 0.0001, 'variance': 39.3462}
{'A1': 0.602, 'Lneck': 0.1376, 'meanDneck_trunc70pc': 0.0273, 'maxDhead': 0.014, 'Dss': 0.0035, 'Lhead': 0.0002}
{'A1': 0.602, 'Lneck': 0.3457, 'meanDneck_trunc70pc': 0.105, 'maxDhead': 0.0604, 'Dss': 0.0162, 'Lhead': 0.0009}
**********V sp/dend**********
{'Lneck': 0.5621, 'meanDneck_trunc70pc': 0.0954, 'maxDhead': 0.0146, 'A1': 0.0016, 'Dss': 0.0003, 'Lhead': 0.0, 'variance': 2.3418}
{'Lneck': 0.5621, 'meanDneck_trunc70pc': 0.0779, 'A1': 0.0112, 'm

# Analysis timed E/I

In [19]:
with open(folder+"dis_ga_glu02v3_1000_70_65sh.pickle","rb") as f:
    dis,mes1,mes01 = pickle.load(f)

with open(folder+"dis_glu_ga02v3_1000_70_65sh.pickle","rb") as f:
    dis,mes,mes0,times,times0 = pickle.load(f)

labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = labelsmod
labels.append('A2')
datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddataT = pd.DataFrame(data=datt,columns=labels)

print("\nGABA-GLU (-2ms)----------------------------------------\n")
print('*'*10+'V in spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1]))
print('*'*10+'V in dend shaft'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2]))
print('*'*10+'Calcium'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4]))

print("\nGLU-GABA (2ms)----------------------------------------\n")
#print('*'*10+'V in spine head'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,1]/mes0[:,1]))
#print(LinReg_varexplain_F(pddataT,mes[:,1]/mes0[:,1]))
#print('*'*10+'V in dend shaft'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,2]/mes0[:,2]))
#print(LinReg_varexplain_F(pddataT,mes[:,2]/mes0[:,2]))
#print('*'*10+'Calcium'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,4]/mes0[:,4]))
#print(LinReg_varexplain_F(pddataT,mes[:,4]/mes0[:,4]))
print('*'*10+'Time reduction'+10*'*')
print(LinReg_varexplain_B(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0))
# print('*'*10+'Time'+10*'*')
# print(LinReg_varexplain_B(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0))
labels = labelmorph
labels.append('A2')

datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddataT = pd.DataFrame(data=datt,columns=labels)

print("\nGABA-GLU (-2ms)----------------------------------------\n")
print('*'*10+'V in spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1],relative = True))
print('*'*10+'V in dend shaft'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2],relative = True))
print('*'*10+'Calcium'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4],relative = True))

print("\nGLU-GABA (2ms)----------------------------------------\n")
#print('*'*10+'V in spine head'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,1]/mes0[:,1]))
#print(LinReg_varexplain_F(pddataT,mes[:,1]/mes0[:,1]))
#print('*'*10+'V in dend shaft'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,2]/mes0[:,2]))
#print(LinReg_varexplain_F(pddataT,mes[:,2]/mes0[:,2]))
#print('*'*10+'Calcium'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,4]/mes0[:,4]))
#print(LinReg_varexplain_F(pddataT,mes[:,4]/mes0[:,4]))
print('*'*10+'Time reduction'+10*'*')
print(LinReg_varexplain_B(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0,relative = True))
# print('*'*10+'Time'+10*'*')
# print(LinReg_varexplain_B(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0,relative = True))



GABA-GLU (-2ms)----------------------------------------

**********V in spine head**********
{'A2': 0.399, 'Rneck': 0.5545, 'A1': 0.0001, 'Dss': 0.0, 'variance': 0.0029}
{'A2': 0.399, 'Rneck': 0.5333, 'Dss': 0.0014, 'A1': 0.0002}
**********V in dend shaft**********
{'Rneck': 0.3691, 'A2': 0.5385, 'A1': 0.0069, 'Dss': 0.0032, 'variance': 0.0044}
{'Rneck': 0.3691, 'A2': 0.518, 'Dss': 0.0099, 'A1': 0.0006}
**********Calcium**********
{'A1': 0.2738, 'Rneck': 0.2235, 'A2': 0.0187, 'Dss': 0.0085, 'variance': 0.0083}
{'A1': 0.2738, 'Rneck': 0.2102, 'A2': 0.0253, 'Dss': 0.0123}

GLU-GABA (2ms)----------------------------------------

**********Time reduction**********
{'A2': 0.6414, 'Dss': 0.2291, 'Rneck': 0.0307, 'A1': 0.0258, 'variance': 0.0061}
{'A2': 0.6414, 'Dss': 0.2207, 'Rneck': 0.036, 'A1': 0.0162}

GABA-GLU (-2ms)----------------------------------------

**********V in spine head**********
{'A2': 0.399, 'Lneck': 0.2888, 'meanDneck_trunc70pc': 0.07, 'Lhead': 0.0277, 'Dss': 0.0056, 'ma

In [27]:
y = times/times0
X = pddataT.copy()
labels = labelmorph

idcs = arange(X.shape[1]).tolist()
vardat = zeros((X.shape[1],2))

y_predt = y*0.0
var0 = var(y)
print(var0)
minvar = var0
for k in range(X.shape[1]-1):
    varts = zeros(len(idcs))
    for iv,i in enumerate(idcs):
        regressor.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = regressor.predict(X.values[:,i:(i+1)])
        vart = ((y-y_predt-y_pred)).var()
        varts[iv] = vart
        print(i, vart)
    sel = argmin(varts)
    print('min', sel)
    i = idcs[sel]
    regressor.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = regressor.predict(X.values[:,i:(i+1)])
    y_predt += y_pred

    vardat[k,:] = (idcs[sel],varts[sel])
    idcs.pop(sel)

i = idcs[0]
regressor.fit(X.values[:,i:(i+1)], y-y_predt)
y_pred = regressor.predict(X.values[:,i:(i+1)])
vart = ((y-y_predt-y_pred)).var()
vardat[-1,:] = (idcs[0],vart)

0.006170174872096131
0 0.004904410723351957
1 0.0037716463034893634
2 0.006112621827601585
3 0.006145191588817245
4 0.006104275600116639
5 0.004957798731040344
6 0.0022125830832433343
7 0.0022125830832433343
min 6
0 0.0018976055851693395
1 0.0008505487968895113
2 0.0020714756497560794
3 0.002075914651341398
4 0.0020137502213632245
5 0.0019532901289316734
7 0.0022125830832433347
min 1
0 0.0008256376767254139
2 0.0007180586954959487
3 0.0008458926866195774
4 0.0006658831638107062
5 0.0008098745151219856
7 0.0008004183589109603
min 3
0 0.0005770105754599648
2 0.0006549797740667575
3 0.0006517684171393513
5 0.0005578019510253695
7 0.0006323834702571697
min 3
0 0.0005560964428368632
2 0.0005187145539067963
3 0.0005485120678293822
7 0.0004789308207277832
min 3
0 0.0004643153772843929
2 0.0004470532936208261
3 0.0004745074094145297
min 1
0 0.00041682186428408473
3 0.00043514722920568214
min 0


In [39]:
result = {}

var0 = var(y)
vsa = 1-(vardat[0,1])/var0
result[labels[int(vardat[0,0])]] = int(10000*(vsa))/10000
print(vsa)
print(labels[int(vardat[0,0])])
for i in range(1,vardat.shape[0]):
    vs = 1-(vardat[i,1])/var0
    result[labels[int(vardat[i,0])]]=int(10000*(vs-vsa))/10000
    print(vs-vsa)
    vsa = vs
    print(labels[int(vardat[i,0])])
print(result)

0.6414067463064177
A2
0.22074484347493273
Dss
0.02992875192467115
meanDneck_trunc70pc
0.017516717925471048
A1
0.012782641000058437
A2
0.005166389570434915
Lneck
0.004899606569249593
maxDhead
0.0015691166661835343
Lhead
{'A2': 0.0127, 'Dss': 0.2207, 'meanDneck_trunc70pc': 0.0299, 'A1': 0.0175, 'Lneck': 0.0051, 'maxDhead': 0.0048, 'Lhead': 0.0015}


In [ ]:
from sklearn.feature_selection import SelectKBest,VarianceThreshold,SelectFromModel, chi2, f_regression

from sklearn.linear_model import LassoCV


In [ ]:
varthr = VarianceThreshold()
X = StandardScaler().fit_transform(datt)

varthr.fit_transform(datt)
yrt = mesT[:,3]

X_kbest=SelectKBest(score_func=f_regression,k=4).fit_transform(pddataT,yrt)


In [ ]:
print(X_kbest)
pddataT.head()

In [ ]:
regressor = LinearRegression()  
regressor.fit(pddataT, yrt)
y_pred = regressor.predict(datt)

sfm = SelectFromModel(regressor, threshold=1e-1)
sfm.fit(pddataT,yrt)
n_features = sfm.transform(pddataT).shape[1]

In [ ]:
X_transform = sfm.transform(pddataT)
print(n_features)
plt.title(
    " "
    "threshold %0.3f." % sfm.threshold)
feature1 = X_transform[:, 0]
feature2 = X_transform[:, 1]
scatter(feature1, feature2, c = yrt,alpha=0.8)
colorbar()
plt.xlabel("Feature number 1")
plt.ylabel("Feature number 2")
plt.ylim([np.min(feature2), np.max(feature2)])


In [ ]:
pddataT.corr()

In [ ]:
labels,varthr.variances_/varthr.variances_.sum()

In [ ]:
i = 1
regressor.fit(datt[:,i:(i+1)], mesT[:,3])
y_pred0 = regressor.predict(datt[:,i:(i+1)])
var0 = ((mesT[:,3]-y_pred0)).var()
print(var0)
i = 3
regressor.fit(datt[:,i:(i+1)], mesT[:,3]-y_pred0)
y_pred1 = regressor.predict(datt[:,i:(i+1)])
var1 = ((mesT[:,3]-y_pred0-y_pred1)).var()
y_predt = y_pred0+y_pred1
for i in [0,2,4,5,6]:
    regressor.fit(datt[:,i:(i+1)], mesT[:,3]-y_predt)
    y_pred2 = regressor.predict(datt[:,i:(i+1)])
    var2 = ((mesT[:,3]-y_predt-y_pred2)).var()
    print(labels[i],var2,var(mesT[:,3]))

In [ ]:
# #############################################################################
# Generate random sample data
centers = [[1, 1,5,2], [-1, -1,3,4], [1, -1,2,4]]
X, _ = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

# Change X to real statistical data for spines
file = "spine_stats.dat"
X = load(file)


In [ ]:
X = StandardScaler().fit_transform(X)
X.shape

In [ ]:
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#      % metrics.adjusted_rand_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
# #############################################################################
# Plot result
from matplotlib.pylab import cm
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [cm.Spectral(each) for each in linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

title('Estimated number of clusters: %d' % n_clusters_)
show()